In [ ]:
import numpy as np



# 写卷积函数
def convolve(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    """
Convolve an image with a kernel assuming zero-padding of the image to handle the borders
:param image: the image (either greyscale shape=(rows,cols) or colour shape=(rows,cols,channels)) :type numpy.ndarray
:param kernel: the kernel (shape=(kheight,kwidth); both dimensions odd) :type numpy.ndarray
:returns the convolved image (of the same shape as the input image)
:rtype numpy.ndarray
"""
# Your code here. You'll need to vectorise your implementation to ensure it runs # at a reasonable speed.

    [h,w] = [image.shape[0],image.shape[1]] #图片边长
    [k,_] = kernel.shape
    r = int(k/2)

    #定义边界填充0后的map
    padding_im = np.zeros([h+2*r, w+2*r])
    #store result
    rs = np.zeros([h,w])
    #将输入在指定该区域赋值，即除了4个边界后，剩下的区域
    padding_im[r:-r, r:-r] = image

    #对每个点为中心的区域遍历
    for i in range(h):
        for j in range(w):
            #取出当前点为中新的k*k区域
            rs[i, j] = np.sum(padding_im[i: i+k, j: j+k]*kernel)
    return rs #convolved image


In [ ]:
# coding=UTF-8
import numpy as np
from MyConvolution import convolve
#图片加载转换成矩阵

def myHybridImages(lowImage: np.ndarray, lowSigma: float, highImage: np.ndarray, highSigma: float) -> np.ndarray:
    """
Create hybrid images by combining a low-pass and high-pass filtered pair.
:param lowImage: the image to low-pass filter (either greyscale shape=(rows,cols)
    or colour shape=(rows,cols,channels))
:type numpy.ndarray
:param lowSigma: the standard deviation of the Gaussian used for low-pass filtering
    lowImage :type float
:param highImage: the image to high-pass filter (either greyscale shape=(rows,cols)
    or colour shape=(rows,cols,channels))
:type numpy.ndarray
:param highSigma: the standard deviation of the Gaussian used
    for low-pass filtering highImage before subtraction to create the high-pass filtered image
:type float
:returns returns the hybrid image created
by low-pass filtering lowImage with a Gaussian of s.d. lowSigma and combining it with
a high-pass image created by subtracting highImage from highImage convolved with
a Gaussian of s.d. highSigma. The resultant image has the same size as the input images.
:rtype numpy.ndarray
    """
    #低频
    low_kernel = makeGaussianKernel(lowSigma)
    if (lowImage.shape[2]==3):
        lowIm_r = convolve(lowImage[:, :, 0], low_kernel)
        lowIm_g = convolve(lowImage[:, :, 1], low_kernel)
        lowIm_b = convolve(lowImage[:, :, 2], low_kernel)

        lowIm_con = np.dstack([lowIm_r, lowIm_g, lowIm_b])
    else:
        lowIm_con = convolve(lowImage, low_kernel)


    #高频
    high_kernel = makeGaussianKernel(highSigma)
    if (highImage.shape[2]==3):
        highIm_r = highImage[:, :, 0] - convolve(highImage[:, :, 0], high_kernel)
        highIm_g = highImage[:, :, 1] - convolve(highImage[:, :, 1], high_kernel)
        highIm_b = highImage[:, :, 2] - convolve(highImage[:, :, 2], high_kernel)

        highIm_con = np.dstack([highIm_r, highIm_g, highIm_b])
    else:
        highIm_con = highImage-convolve(highImage, high_kernel)

    #融合
    hybrid_res = lowIm_con + highIm_con

    return hybrid_res



# Your code here.
def makeGaussianKernel(sigma: float) -> np.ndarray:
    """
    Use this function to create a 2D gaussian kernel with standard deviation sigma.
    The kernel values should sum to 1.0,
    and the size should be floor(8*sigma+1)
    or floor(8*sigma+1)+1 (whichever is odd) as per the assignment specification.
    """
    # Your code here.
    # size 对应卷积kernel大小

    size = int(8.0 * sigma + 1.0)  # (this implies the window is +/- 4 sigmas from the centre of the Gaussian)

    if (size % 2 == 0):
        # size must be odd
        size = size + 1
    kernel_size =size

    kernel = np.zeros([kernel_size,kernel_size])
    center_kernel = kernel_size//2
    s = 2*(sigma**2)

    sum_val = 0
    for i in range(kernel_size):
        for j in range(kernel_size):
            x = i-center_kernel
            y = j-center_kernel
            kernel[i,j] = np.exp(-(x**2+y**2)/s)
            sum_val += kernel[i,j]

    sum_val = 1/sum_val
    kernel_res = kernel * sum_val

    return kernel_res